In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

with open("/kaggle/input/pride-prejudice-subtitles-and-text/PP.txt", "r", encoding="utf8") as f:
    data = f.read()

# Prétraitement du texte
text = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', '').replace('”', '')
text = ' '.join(text.split())

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

# Enregistrer le tokenizer
with open('token.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

vocab_size = len(tokenizer.word_index) + 1
print(f"Taille du vocabulaire : {vocab_size}")



Taille du vocabulaire : 6745


In [ ]:
sequence_data = tokenizer.texts_to_sequences([text])[0]
sequences = []
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

sequences = np.array(sequences)



In [ ]:
# Diviser les données en entrée et sortie
X = sequences[:, :-1]  # Les 5 premiers mots
y = sequences[:, -1]   # Le mot à prédire
y = to_categorical(y, num_classes=vocab_size)





In [2]:
# Définir le modèle
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(150))
model.add(Dense(vocab_size, activation='softmax'))
model.build(input_shape=(None, X_train.shape[1]))  # Définir la forme d'entrée en fonction de X_train

print(model.summary())



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 3, 100)         │       674,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6745)           │     1,018,495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,843,595 (7.03 MB)

 Trainable params: 1,843,595 (7.03 MB)

 Non-trainable params: 0 (0.00 B)

None


In [3]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_val, y_val))



Epoch 1/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 36s 9ms/step - accuracy: 0.0452 - loss: 6.5457
Epoch 2/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 34s 9ms/step - accuracy: 0.1145 - loss: 5.4824
Epoch 3/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - accuracy: 0.1425 - loss: 5.0417
Epoch 4/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - accuracy: 0.1604 - loss: 4.7355
Epoch 5/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - accuracy: 0.1769 - loss: 4.4555
Epoch 6/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 34s 9ms/step - accuracy: 0.1958 - loss: 4.1962
Epoch 7/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 34s 9ms/step - accuracy: 0.2203 - loss: 3.9407
Epoch 8/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - accuracy: 0.2465 - loss: 3.7179
Epoch 9/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - accuracy: 0.2742 - loss: 3.5061
Epoch 10/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - accuracy: 0.3069 - loss: 3.2898
Epoch 11/100
3809/3809 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - accuracy: 0.3372 - loss: 3.1116
Epoch 12

In [27]:
input_text = "i want to"
predict_next_words = 34


for _ in range(predict_next_words):
    input_sequence = tokenizer.texts_to_sequences([input_text])[0]
    
    input_sequence = input_sequence[-3:]
    
    input_sequence = np.array(input_sequence).reshape(1, -1)
    
    # Prédire le mot suivant
    predicted_prob = model.predict(input_sequence, verbose=0)
    predicted_index = np.argmax(predicted_prob, axis=-1)
    
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    
    if output_word == "":
        print("Aucun mot prédit trouvé.")
        break
    
    input_text += " " + output_word

print(input_text)

i want to speak to elizabeth privately of what lydia had let fall elizabeth was glad to be certain that had been his motive what can have induced him to behave so cruelly a thorough determined dislike
